# Stations Discovery + Match-Up

Given a SENTINEL-3 OLCI, i.e. [S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004](https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004) with specified attributes:

- `geometry`,
- [`properties.start_datetime`, `properties.end_datetime`]

search all active AERONET stations, where:

- `Site_Latitude(Degrees),Site_Longitude(Degrees)` in `geometry`

for each discovered station, filter all acquisitions where:

- `properties.start_datetime` - 1h <= `Date(dd:mm:yyyy),Time(hh:mm:ss)` <= `properties.end_datetime` + 1h

In [1]:
from httpx import (
    Client,
    Response
)
from pathlib import Path
from pystac import Item
from typing import Any

import json
import sys

out_dir: Path = Path('.')

with Client() as client:
    response: Response = client.get('https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004')
    item_str: str = response.text
    item_dict = json.loads(item_str)

json.dump(item_dict, sys.stdout, indent=2)

{
  "id": "S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004",
  "bbox": [
    -1.51876,
    39.545,
    18.484,
    52.4535
  ],
  "type": "Feature",
  "links": [
    {
      "rel": "collection",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc"
    },
    {
      "rel": "parent",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc"
    },
    {
      "rel": "root",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/"
    },
    {
      "rel": "self",
      "type": "application/geo+json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004"
    },
    {
      "rel": "version-history",
      "href": "https://trace.

## Execute the `query_stations_from_parquet` operation

In [2]:

source_item: Item = Item.from_dict(item_dict)
cql2_filter = {
    "op": "and",
    "args": [
        {
            "op": "s_intersects",
            "args": [
                {"property": "geometry"},
                source_item.geometry,
            ],
        }
    ],
}

json.dump(cql2_filter, sys.stdout, indent=2)

{
  "op": "and",
  "args": [
    {
      "op": "s_intersects",
      "args": [
        {
          "property": "geometry"
        },
        {
          "type": "Polygon",
          "coordinates": [
            [
              [
                -1.5187599999999999,
                41.97070000000001
              ],
              [
                13.725899999999998,
                39.545
              ],
              [
                18.484,
                49.88549999999999
              ],
              [
                0.10018799999999999,
                52.45349999999999
              ],
              [
                -1.5187599999999999,
                41.97070000000001
              ]
            ]
          ]
        }
      ]
    }
  ]
}

In [3]:
from pygeofilter_aeronet import (
    DEFAULT_STATIONS_PARQUET_URL,
    query_stations_from_parquet
)
from typing import List

_, resulting_items = query_stations_from_parquet(
    cql2_filter=cql2_filter,
    file_path=DEFAULT_STATIONS_PARQUET_URL
)

## Prepare the Map screen

In [4]:
print(f"Found {len(resulting_items)} STAC Item(s) representing AERONET Station(s)")

Found 114 STAC Item(s) representing AERONET Station(s)


In [5]:
from folium import (
    GeoJson,
    LayerControl,
    Map
)
from folium.plugins import Fullscreen

map: Map = Map()
layer_control = LayerControl(position="topright", collapsed=True)
fullscreen = Fullscreen()

GeoJson(
    source_item,
    name=source_item.id,
    style_function=lambda f: {
        "fillColor": "yellow",
        "color": "red",
        "weight": 2,
        "fillOpacity": 0.3,
    }
).add_to(map)

for item in resulting_items:
    GeoJson(
        item,
        name=item.id,
        style_function=lambda f: {
            "fillColor": "yellow",
            "color": "red",
            "weight": 2,
            "fillOpacity": 0.3,
        }
).add_to(map)

layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds()) # type: ignore not to important for the demo
map

## `search` data for each discovered station

In [ ]:
from datetime import (
    datetime,
    timedelta,
    timezone
)
from dateutil import parser
from geopandas import read_parquet
from geopandas.geodataframe import GeoDataFrame
from IPython.display import (
    display,
    Markdown
)
from pygeofilter_aeronet import aeronet_search

def _parse_date_from_properties(date_string_id: str) -> datetime:
    return parser.parse(source_item.properties[date_string_id])

start_datetime = _parse_date_from_properties('start_datetime') - timedelta(hours=1)
end_datetime = _parse_date_from_properties('end_datetime') + timedelta(hours=1)

def _serialize_date(input_date: datetime) -> str:
    dt_utc = input_date.astimezone(timezone.utc)
    return dt_utc.isoformat().replace("+00:00", "Z")

map: Map = Map()
layer_control = LayerControl(position="topright", collapsed=True)
fullscreen = Fullscreen()

GeoJson(
    source_item,
    name=source_item.id,
    style_function=lambda f: {
        "fillColor": "yellow",
        "color": "red",
        "weight": 2,
        "fillOpacity": 0.3,
    }
).add_to(map)

DEFAULT_TIMEOUT: int = 5

FILTERING_SITES: List[str] = [ 'Lille', 'Paris', 'Rome_Tor_Vergata' ]

for item in resulting_items:
    site_name: str = item.properties['aeronet:site_name']

    if site_name in FILTERING_SITES:
        cql2_filter = {
            "op": "and",
            "args": [
                {"op": "eq", "args": [{"property": "site"}, site_name]},
                {"op": "eq", "args": [{"property": "data_type"}, "AOD10"]},
                {"op": "eq", "args": [{"property": "format"}, "csv"]},
                {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
                {
                    "op": "t_after",
                    "args": [
                        {"property": "time"},
                        {"timestamp": _serialize_date(start_datetime)},
                    ],
                },
                {
                    "op": "t_before",
                    "args": [
                        {"property": "time"},
                        {"timestamp": _serialize_date(end_datetime)},
                    ],
                }
            ],
        }

        try:
            resulting_item: Item = aeronet_search(
                cql2_filter=cql2_filter,
                output_dir=out_dir,
                timeout=DEFAULT_TIMEOUT
            )

            display(Markdown(f"### {site_name} results as Data Frame"))

            geoparquet_file: str = resulting_item.get_assets()['geoparquet'].href
            geoparquet_data: GeoDataFrame = read_parquet(geoparquet_file)

            display(geoparquet_data)

            geoparquet_data = geoparquet_data.dissolve(by='AERONET_Site')

            GeoJson(
                geoparquet_data.to_json(default=str),
                name="Stac Item footprints",
                style_function=lambda x: {"fillColor": "#00000000", "color": "#0000ff", "weight": 1},
                control=True,
            ).add_to(map)
        except Exception as e:
            print(f"An error occurred while retrieving data from site {site_name}: {e}")

layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds()) # type: ignore not to important for the demo
map

An error occurred while retrieving data from site La_Crau: No columns to parse from file


2025-11-17 18:08:42.411 | SUCCESS  | pygeofilter_aeronet:aeronet_search:244 - Query on https://aeronet.gsfc.nasa.gov successfully obtained data:
2025-11-17 18:08:42.416 | SUCCESS  | pygeofilter_aeronet:aeronet_search:257 - Data saved to to CSV file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/8b5c3413-e5e1-4553-ba64-48f731263c84.csv
2025-11-17 18:08:42.430 | SUCCESS  | pygeofilter_aeronet:aeronet_search:291 - Data saved to GeoParquet file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/8b5c3413-e5e1-4553-ba64-48f731263c84.parquet


### Lille results as Data Frame

,AERONET_Site,Date(dd:mm:yyyy),Time(hh:mm:ss),Day_of_Year,AOD_1640nm,AOD_1020nm,AOD_870nm,AOD_865nm,AOD_779nm,AOD_675nm,...,N[340-440_Angstrom_Exponent],N[440-675_Angstrom_Exponent[Polar]],Data_Quality_Level,AERONET_Instrument_Number,AERONET_Site_Name,Site_Latitude(Degrees),Site_Longitude(Degrees),Site_Elevation(m),geometry,datetime
0,Lille,30:10:2025,12:00:00,303,0.294629,0.309954,0.303359,-999.0,-999.0,0.296156,...,45,0,lev10,1345,Lille,50.611667,3.141667,60.0,POINT (3.14167 50.61167),2025-10-30 12:00:00
1,Lille,31:10:2025,12:00:00,304,0.546914,0.518839,0.499880,-999.0,-999.0,0.486531,...,5,0,lev10,1345,Lille,50.611667,3.141667,60.0,POINT (3.14167 50.61167),2025-10-31 12:00:00


An error occurred while retrieving data from site JUNGFRAU: No columns to parse from file
An error occurred while retrieving data from site Aire_Adour: No columns to parse from file
An error occurred while retrieving data from site Venise: No columns to parse from file


...

An error occurred while retrieving data from site BORDEAUX: No columns to parse from file
An error occurred while retrieving data from site Marseille: No columns to parse from file
An error occurred while retrieving data from site Realtor: No columns to parse from file
An error occurred while retrieving data from site Vinon: No columns to parse from file
An error occurred while retrieving data from site Oostende: No columns to parse from file


2025-11-17 18:09:21.322 | SUCCESS  | pygeofilter_aeronet:aeronet_search:244 - Query on https://aeronet.gsfc.nasa.gov successfully obtained data:
2025-11-17 18:09:21.330 | SUCCESS  | pygeofilter_aeronet:aeronet_search:257 - Data saved to to CSV file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/53c20378-6367-4edc-9414-a12d8dc0a60e.csv
2025-11-17 18:09:21.340 | SUCCESS  | pygeofilter_aeronet:aeronet_search:291 - Data saved to GeoParquet file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/53c20378-6367-4edc-9414-a12d8dc0a60e.parquet


### Davos results as Data Frame

,AERONET_Site,Date(dd:mm:yyyy),Time(hh:mm:ss),Day_of_Year,AOD_1640nm,AOD_1020nm,AOD_870nm,AOD_865nm,AOD_779nm,AOD_675nm,...,N[340-440_Angstrom_Exponent],N[440-675_Angstrom_Exponent[Polar]],Data_Quality_Level,AERONET_Instrument_Number,AERONET_Site_Name,Site_Latitude(Degrees),Site_Longitude(Degrees),Site_Elevation(m),geometry,datetime
0,Davos,30:10:2025,12:00:00,303,0.062397,0.060122,0.058615,-999.0,-999.0,0.057144,...,76,0,lev10,1144,Davos,46.81281,9.84369,1589.0,POINT (9.84369 46.81281),2025-10-30 12:00:00
1,Davos,31:10:2025,12:00:00,304,0.179512,0.131166,0.126053,-999.0,-999.0,0.120485,...,27,0,lev10,1144,Davos,46.81281,9.84369,1589.0,POINT (9.84369 46.81281),2025-10-31 12:00:00


An error occurred while retrieving data from site Tarbes: No columns to parse from file
An error occurred while retrieving data from site Pic_du_midi: The read operation timed out
An error occurred while retrieving data from site Munich_University: The read operation timed out
An error occurred while retrieving data from site Tarbes_Etal: No columns to parse from file
An error occurred while retrieving data from site ISDGM_CNR: No columns to parse from file
An error occurred while retrieving data from site Fontainebleau: _ssl.c:1015: The handshake operation timed out
An error occurred while retrieving data from site Munich_Maisach: [Errno 111] Connection refused
An error occurred while retrieving data from site Villefranche: [Errno 111] Connection refused
An error occurred while retrieving data from site Carpentras: [Errno 111] Connection refused
An error occurred while retrieving data from site Rossfeld: [Errno 111] Connection refused
An error occurred while retrieving data from site 